In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict, cross_val_score


from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

import seaborn as sns

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
f9 = r'/home/william/Dropbox/Kaggle/Soccer/season9.csv'
f10 = r'/home/william/Dropbox/Kaggle/Soccer/season10.csv'
f11 = r'/home/william/Dropbox/Kaggle/Soccer/season11.csv'
f12 = r'/home/william/Dropbox/Kaggle/Soccer/season12.csv'
f13 = r'/home/william/Dropbox/Kaggle/Soccer/season13.csv'
f14 = r'/home/william/Dropbox/Kaggle/Soccer/season14.csv'
f15 = r'/home/william/Dropbox/Kaggle/Soccer/season15.csv'
f16 = r'/home/william/Dropbox/Kaggle/Soccer/season16.csv'


df9 = pd.read_csv(f9)
df10 = pd.read_csv(f10)
df11 = pd.read_csv(f11)
df12 = pd.read_csv(f12)
df13 = pd.read_csv(f13)
df14 = pd.read_csv(f14)
df15 = pd.read_csv(f15)
df16 = pd.read_csv(f16)

df_train = df9.append([df10,df11,df12,df13,df14,df15],ignore_index=True)

train_X = df_train.drop('Home/Away win',axis=1)
train_y = df_train['Home/Away win']
train_y = (train_y==0)+0

test_X = df16.drop('Home/Away win',axis=1)
test_y = df16['Home/Away win']
test_y = (test_y==0)+0



#Adding a feature to indicate extra likelihood of draw
draw_NB = GaussianNB()
draw_NB.fit(train_X,train_y)

predicted_p = draw_NB.predict_proba(train_X)[:,1]
indices = predicted_p.argsort()
indices = indices[-np.sum(train_y):]
train_X['draw_maybe'] = 0
train_X.iloc[indices,-1] = 1 

predicted_p = draw_NB.predict_proba(test_X)[:,1]
indices = predicted_p.argsort()
no_draw = int(np.average(train_y)*test_y.size)
indices = indices[-no_draw:]
test_X['draw_maybe'] = 0
test_X.iloc[indices,-1] = 1

In [15]:
print(1-np.average(train_y))
print(1-np.average(test_y))


0.742105263158
0.778947368421


In [17]:
GNB = GaussianNB()
GNB.fit(train_X,train_y)
GNB.score(test_X,test_y)

0.69999999999999996

In [21]:
#RFC
RFC = RandomForestClassifier()
RFC.fit(train_X,train_y)
RFC.predict(test_X)

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0,